<a href="https://colab.research.google.com/github/Antique-1/llm-programming1/blob/main/11_04_1.2_%EB%9E%AD%EC%B2%B4%EC%9D%B8_%EC%B1%97%EB%B4%87_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 랭체인 멀티턴

- LangChain의 RunnableWithMessageHistory를 사용하여 대화기록을 관리하는 챗봇 구현
- 세션 ID별로 대화 기록이 어떻게 분리되어 저장되는지 확인

## 라이브러리 불러오기

In [2]:
%pip install langchain langchain_openai

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.9 MB/s eta 0:00:00


In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

## API KEY 불러오기

In [4]:
from openai import OpenAI
import os

In [ ]:
load_dotenv()

True

## 대화 기록 함수 정의

In [6]:
api_key = ""
client = OpenAI(api_key=api_key)

In [7]:
model = ChatOpenAI(
    model="gpt-4o-mini",
    api_key = api_key
 )

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)

## 첫 번째 대화

In [10]:
config = {"configurable": {"session_id": "abc2"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 김도현이야.")],
    config=config,
)

print(response.content)

안녕하세요, 김도현님! 어떻게 도와드릴까요?


## 두 번째 대화

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 김도현입니다. 다른 질문이나 궁금한 점이 있으신가요?


## 세션 테스트

In [12]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'죄송하지만, 당신의 이름은 알 수 없습니다. 하지만 도움이 필요하시거나 질문이 있다면 언제든지 말씀해 주세요!'

In [13]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'우리는 처음에 당신이 김정은이라고 소개하셨고, 그 후에 이름이 김도현이라고 하셨습니다. 이후에 당신의 이름에 대해 질문하셨고, 제가 당신의 이름을 확인해드렸습니다. 더 궁금한 점이나 이야기하고 싶은 내용이 있으면 말씀해 주세요!'

## 스트리밍 응답 출력

In [14]:
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="")
    # print(r.content, end="|")

김도현님이라고 말씀하신 것만으로는 어느 나라 사람인지 정확히 맞추기 어렵습니다. 이름이 한국에서 흔하게 사용되기 때문에 한국인일 가능성이 높다고 추측할 수 있습니다. 

한국의 문화에 대해 말씀드리자면:

1. **음식**: 한국 cuisine은 다양하고 풍부합니다. 대표적인 음식으로는 김치, 비빔밥, 불고기 등이 있으며, 각 지역마다 특색 있는 음식이 존재합니다.

2. **전통**: 한국은 오랜 전통 문화를 가지고 있습니다. 한복은 전통적인 의상으로, 특별한 날이나 명절에 입습니다.

3. **가족 중심**: 한국 사회는 가족과 공동체의 중요성을 강조합니다. 명절이나 중요한 행사 때에는 가족이 함께 모여 격식을 갖추는 경우가 많습니다.

4. **K-POP과 한류**: 최근 몇 년 사이에 K-POP과 한국 드라마, 영화가 세계적으로 큰 인기를 끌고 있습니다. 이는 한국 문화의 글로벌한 확산에 큰 도움을 주고 있습니다.

5. **명절**: 설날(새해)과 추석(가을 수확제)과 같은 명절이 있으며, 이때는 전통적인 놀이, 음식, 제사를 통해 조상을 기리는 풍습이 있습니다.

이러한 문화는 한국의 독특한 정체성을 형성하고 있으며, 세계 여러 나라 사람들에게도 사랑받고 있습니다. 더 궁금한 점이나 다른 주제에 대해 이야기하고 싶으시면 말씀해 주세요!